In [8]:
!pip install --upgrade langchain langchain-core langgraph dotenv -qU langchain-groq -U langchain-community faiss-cpu chromadb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 47.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 73.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 79.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [34]:
from typing import TypedDict, List, Optional, Mapping, Any, Union, Annotated, Sequence, Dict
from langchain_core.messages import HumanMessage, AIMessage, tool, BaseMessage, ToolMessage, SystemMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain.tools.render import render_text_description
import requests
from langchain.memory import ConversationBufferMemory
from kaggle_secrets import UserSecretsClient
import os
from langchain_groq import ChatGroq
import sqlite3 as sql
import threading
import time

# Add imports for RAG and multimodal capabilities
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import WikipediaLoader
from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from PIL import Image
import base64
import io
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
import numpy as np
import pickle
import json

# Initialize memory
langchain_memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Initialize secrets
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("groq_api_key")
secret_value_1 = user_secrets.get_secret("liteApi_key")

os.environ["GROQ_API_KEY"] = secret_value_0

# Initialize LLM
llm = ChatGroq(temperature=0, model_name="qwen/qwen3-32b")

# Initialize embeddings for vector search
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

# Initialize image captioning model for multimodal input
device = "cuda" if torch.cuda.is_available() else "cpu"
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

# Thread-safe database connection
thread_local = threading.local()

def get_db_connection():
    """Get a thread-safe database connection"""
    if not hasattr(thread_local, 'connection'):
        db_path = 'flight.sqlite'
        thread_local.connection = sql.connect(db_path, timeout=30)
        thread_local.connection.execute('PRAGMA journal_mode=WAL;')
        thread_local.cursor = thread_local.connection.cursor()
        
        # Create table if it doesn't exist
        thread_local.cursor.execute('''
        CREATE TABLE IF NOT EXISTS flights (
            flight_id INTEGER PRIMARY KEY AUTOINCREMENT,
            flight_number TEXT NOT NULL,
            airline TEXT,
            aircraft TEXT,
            flight_status TEXT,
            departure_airport_iata TEXT,
            departure_airport_name TEXT,
            departure_airport_city TEXT,
            departure_airport_country TEXT,
            departure_airport_timezone TEXT,
            destination_airport_iata TEXT,
            destination_airport_name TEXT,
            destination_airport_city TEXT,
            destination_airport_country TEXT,
            destination_airport_timezone TEXT,
            scheduled_departure DATETIME,
            scheduled_arrival DATETIME,
            created_timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
        ''')
        thread_local.connection.commit()
    
    return thread_local.connection, thread_local.cursor

# Function to load Wikipedia data
def load_wikipedia_travel_data():
    """Load travel-related Wikipedia articles"""
    # List of travel-related Wikipedia pages to load
    travel_topics = [
        # Indian cities and tourist destinations
        "Tourism_in_India",
        "Delhi",
        "Mumbai",
        "Bangalore",
        "Chennai",
        "Kolkata",
        "Hyderabad",
        "Goa",
        "Kerala",
        "Rajasthan",
        "Taj_Mahal",
        "Jaipur",
        "Agra",
        "Varanasi",
        
        # International destinations
        "Paris",
        "London",
        "New_York_City",
        "Tokyo",
        "Dubai",
        "Singapore",
        "Bangkok",
        "Barcelona",
        
        # Travel-related topics
        "Visa_policy_of_India",
        "List_of_airlines_of_India",
        "Indian_Railways",
        "Backpacking_(travel)",
        "Travel_insurance",
        "Tourism",
        
        # Airports
        "Indira_Gandhi_International_Airport",
        "Chhatrapati_Shivaji_Maharaj_International_Airport",
        "Kempegowda_International_Airport",
        "Chennai_International_Airport",
        
        # Travel tips and culture
        "Indian_cuisine",
        "Culture_of_India",
        "Climate_of_India",
        "Languages_of_India"
    ]
    
    all_documents = []
    
    print("Loading Wikipedia articles...")
    for topic in travel_topics:
        try:
            loader = WikipediaLoader(query=topic, load_max_docs=1)
            docs = loader.load()
            all_documents.extend(docs)
            print(f"Loaded: {topic}")
        except Exception as e:
            print(f"Failed to load {topic}: {str(e)}")
    
    return all_documents

# Function to create or load vector store
def initialize_vector_store():
    """Initialize or load the vector store for travel information"""
    vectorstore_path = "travel_wikipedia_vectorstore.pkl"
    
    # Check if pre-built vector store exists
    if os.path.exists(vectorstore_path):
        print("Loading existing vector store...")
        with open(vectorstore_path, 'rb') as f:
            vectorstore = pickle.load(f)
        print("Vector store loaded successfully!")
    else:
        print("Building new vector store from Wikipedia...")
        
        # Load Wikipedia documents
        documents = load_wikipedia_travel_data()
        
        # Also add custom travel information
        custom_docs = [
            # Baggage rules
            "Domestic flight baggage rules in India: Economy class passengers are allowed 15kg check-in baggage and 7kg hand baggage. Business class passengers get 25kg check-in baggage. Excess baggage charges apply at Rs 300-500 per kg.",
            "International baggage allowance varies by airline. Most airlines allow 23kg for economy and 32kg for business class. Hand baggage is typically 7-10kg.",
            
            # Travel tips specific to India
            "Best time to visit North India is October to March. Summer (April-June) can be extremely hot. Monsoon (July-September) brings heavy rainfall.",
            "For South India, October to February is ideal. Kerala and Tamil Nadu have different monsoon patterns.",
            
            # Budget travel tips
            "Budget accommodation in India: Hostels cost Rs 300-800 per night, budget hotels Rs 800-2000, mid-range hotels Rs 2000-5000.",
            "Local transport options: Auto-rickshaws, metro (in major cities), local buses, app-based cabs (Uber, Ola).",
            
            # Safety tips
            "Travel safety in India: Keep copies of important documents, use registered pre-paid taxis from airports, avoid isolated areas after dark.",
            "Health precautions: Drink only bottled water, avoid street food initially, carry basic medicines, get travel insurance.",
            
            # Popular circuits
            "Golden Triangle tour covers Delhi-Agra-Jaipur. It's the most popular tourist circuit taking 5-7 days.",
            "Kerala backwaters tour includes Kochi, Munnar, Thekkady, Alleppey, and Kovalam. Best done in 7-10 days.",
        ]
        
        # Convert custom docs to Document format
        from langchain.schema import Document
        custom_documents = [Document(page_content=doc, metadata={"source": "custom"}) for doc in custom_docs]
        documents.extend(custom_documents)
        
        # Split documents
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
        )
        
        texts = text_splitter.split_documents(documents)
        print(f"Created {len(texts)} text chunks")
        
        # Create vector store
        vectorstore = FAISS.from_documents(texts, embeddings)
        
        # Save vector store for future use
        with open(vectorstore_path, 'wb') as f:
            pickle.dump(vectorstore, f)
        print("Vector store saved successfully!")
    
    return vectorstore

# Initialize vector store
vectorstore = initialize_vector_store()

# Create retrieval chain with different search types
retriever = vectorstore.as_retriever(
    search_type="mmr",  # Maximum Marginal Relevance for diverse results
    search_kwargs={"k": 5, "fetch_k": 10}
)

# Enhanced AgentState to support multimodal input
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    image_context: Optional[str]  # For storing image description

# Global booking dictionaries
hotel_booking = {}
flight_booking = {}

# Image processing function
def process_image(image_data: Union[str, bytes, Image.Image]) -> str:
    """Process image and generate description using BLIP model"""
    try:
        # Handle different input types
        if isinstance(image_data, str):
            # Base64 encoded string
            image_bytes = base64.b64decode(image_data)
            image = Image.open(io.BytesIO(image_bytes))
        elif isinstance(image_data, bytes):
            image = Image.open(io.BytesIO(image_data))
        else:
            image = image_data
        
        # Convert to RGB if necessary
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        # Process with BLIP
        inputs = blip_processor(image, return_tensors="pt").to(device)
        out = blip_model.generate(**inputs, max_length=50)
        caption = blip_processor.decode(out[0], skip_special_tokens=True)
        
        return f"Image shows: {caption}"
    except Exception as e:
        return f"Error processing image: {str(e)}"

# Enhanced RAG tool for general travel queries
@tool
def TravelKnowledgeSearch(query: str) -> str:
    """
    Search comprehensive travel knowledge base (including Wikipedia) for information about destinations, 
    travel tips, visa requirements, cultural information, tourist attractions, etc.
    
    Args:
        query: The travel-related question or topic to search for
        
    Returns:
        Relevant travel information from the knowledge base including Wikipedia sources
    """
    try:
        # Retrieve relevant documents
        relevant_docs = retriever.get_relevant_documents(query)
        
        # Group documents by source
        wikipedia_docs = []
        custom_docs = []
        
        for doc in relevant_docs:
            if doc.metadata.get("source") == "custom":
                custom_docs.append(doc)
            else:
                wikipedia_docs.append(doc)
        
        # Combine the content
        context_parts = []
        
        if wikipedia_docs:
            wiki_content = "\n".join([doc.page_content[:500] for doc in wikipedia_docs[:3]])  # Limit content length
            context_parts.append(f"Wikipedia Information:\n{wiki_content}")
        
        if custom_docs:
            custom_content = "\n".join([doc.page_content for doc in custom_docs])
            context_parts.append(f"Travel Tips:\n{custom_content}")
        
        context = "\n\n".join(context_parts)
        
        # Create a prompt for the LLM to synthesize the information
        prompt = f"""Based on the following travel information, provide a comprehensive answer to the query: {query}

                {context}

                Please provide a helpful, accurate, and well-structured answer that includes:
                1. Key information from reliable sources
                2. Practical tips if applicable
                3. Any important considerations or warnings

                Answer:"""
        
        # Get response from LLM
        response = llm.invoke(prompt)
        
        return response.content
    except Exception as e:
        return f"Error searching travel knowledge: {str(e)}"

# Add a specialized tool for destination information
@tool
def DestinationInfo(destination: str) -> str:
    """
    Get detailed information about a specific travel destination including attractions, 
    best time to visit, local culture, and practical tips.
    
    Args:
        destination: Name of the city, country, or tourist attraction
        
    Returns:
        Comprehensive information about the destination
    """
    try:
        # Search for destination-specific information
        query = f"{destination} tourism attractions culture travel tips best time to visit"
        docs = retriever.get_relevant_documents(query)
        
        # Extract relevant information
        info_sections = {
            "overview": "",
            "attractions": "",
            "best_time": "",
            "culture": "",
            "practical_tips": ""
        }
        
        for doc in docs:
            content = doc.page_content.lower()
            if "attraction" in content or "tourist" in content or "visit" in content:
                info_sections["attractions"] += doc.page_content[:300] + "\n"
            if "climate" in content or "season" in content or "best time" in content:
                info_sections["best_time"] += doc.page_content[:200] + "\n"
            if "culture" in content or "tradition" in content:
                info_sections["culture"] += doc.page_content[:200] + "\n"
        
        # Format the response
        response = f"""Destination Information for {destination}:

            📍 MAIN ATTRACTIONS:
            {info_sections['attractions'][:500] if info_sections['attractions'] else 'Information not available'}
            
            🌤️ BEST TIME TO VISIT:
            {info_sections['best_time'][:300] if info_sections['best_time'] else 'Information not available'}
            
            🎭 CULTURE & TRADITIONS:
            {info_sections['culture'][:300] if info_sections['culture'] else 'Information not available'}
            
            For more specific information, please ask about particular aspects of {destination}."""
        
        return response
    except Exception as e:
        return f"Error getting destination information: {str(e)}"

hotel_booking = {}
flight_booking = {}

@tool
def FlightSearch(departure_airport_iata : Optional[str] = None ,departure_airport_city : Optional[str] = None ,departure_airport_name : Optional[str] = None , destination_airport_name : Optional[str] = None, 
                 destination_airport_iata: Optional[str] = None , destination_airport_city : Optional[str] = None ,
                 departure_time : Optional[str] = None , arrival_time : Optional[str] = None) :
    """
    Search for available flights based on specified criteria.
    
    This function queries the flight database to find flights matching the provided
    search parameters. At least one search parameter must be provided. The function
    supports searching by departure and destination airport details, as well as
    departure and arrival times.
    
    Args:
        departure_airport_iata (Optional[str], optional): 
            IATA code of departure airport (e.g., 'DEL', 'BOM'). Defaults to None.
        departure_airport_city (Optional[str], optional): 
            City name of departure airport (e.g., 'Delhi', 'Mumbai'). Defaults to None.
        departure_airport_name (Optional[str], optional): 
            Full name of departure airport (e.g., 'Indira Gandhi International Airport'). 
            Defaults to None.
        destination_airport_name (Optional[str], optional): 
            Full name of destination airport. Defaults to None.
        destination_airport_iata (Optional[str], optional): 
            IATA code of destination airport. Defaults to None.
        destination_airport_city (Optional[str], optional): 
            City name of destination airport. Defaults to None.
        departure_time (Optional[str], optional): 
            Departure time filter (format: 'YYYY-MM-DD HH:MM:SS'). Defaults to None.
        arrival_time (Optional[str], optional): 
            Arrival time filter (format: 'YYYY-MM-DD HH:MM:SS'). Defaults to None.
    
    Returns:
        list: A list containing column names and flight records matching the search criteria.
              Format: [column_names_list, [flight_record_1, flight_record_2, ...]]
              Returns empty results if no flights match the criteria.
              Returns error message string if an exception occurs.
    
    Raises:
        Exception: Returns error message as string if database query fails or other
                  exceptions occur during execution.
    
    Example:
        >>> # Search flights from Delhi to Mumbai
        >>> results = FlightSearch(departure_airport_iata='DEL', 
        ...                       destination_airport_iata='BOM')
        >>> 
        >>> # Search flights by city names
        >>> results = FlightSearch(departure_airport_city='Delhi', 
        ...                       destination_airport_city='Mumbai')
        >>> 
        >>> # Search flights with specific departure time
        >>> results = FlightSearch(departure_airport_iata='DEL',
        ...                       departure_time='2024-06-07 10:00:00')
    
    Note:
        - The function uses parameterized queries to prevent SQL injection
        - String comparisons are case-sensitive
        - Time filters should be provided in 'YYYY-MM-DD HH:MM:SS' format
        - At least one parameter should be provided for meaningful results
    """
    try:
        connection, cursor = get_db_connection()
        
        # Build WHERE clause conditions and parameters
        conditions = []
        params = []
        
        if departure_airport_iata is not None:
            conditions.append("departure_airport_iata = ?")
            params.append(departure_airport_iata)
            
        if departure_airport_city is not None:
            conditions.append("departure_airport_city = ?")
            params.append(departure_airport_city)
            
        if departure_airport_name is not None:
            conditions.append("departure_airport_name = ?")
            params.append(departure_airport_name)
            
        if destination_airport_name is not None:
            conditions.append("destination_airport_name = ?")
            params.append(destination_airport_name)
            
        if destination_airport_iata is not None:
            conditions.append("destination_airport_iata = ?")
            params.append(destination_airport_iata)
            
        if destination_airport_city is not None:
            conditions.append("destination_airport_city = ?")
            params.append(destination_airport_city)
        
        # If no conditions, return all flights (or you might want to return empty results)
        if not conditions:
            sql_query = "SELECT * FROM flights"
        else:
            sql_query = "SELECT * FROM flights WHERE " + " AND ".join(conditions)
        
        cursor.execute(sql_query, params)
        columns = [description[0] for description in cursor.description]
        results = cursor.fetchall()
        
        return [columns, results[:10]]
            
    except Exception as e:
        return f"Error searching flights: {str(e)}"

@tool
def CancelFlightBooking(flight_number: str):
   """
   Cancel an existing flight booking and remove it from the system.
   
   This function allows users to cancel their previously booked flight tickets by
   providing the flight ID. It checks if a booking exists for the given flight ID
   and removes it from the booking system if found.
   
   Args:
       flight_number (str): The unique identifier of the flight booking to be cancelled.
                       This should match an existing booking in the system.
   
   Returns:
       str: A message indicating the result of the cancellation operation:
            - Success message confirming the booking has been cancelled if the
              flight ID exists in the booking system
            - Error message if no booking is found for the provided flight ID
   
   Example:
       >>> # Cancel an existing booking
       >>> result = CancelFlightBooking("AI101")
       >>> print(result)
       Flight booking with flight Id AI101 is successfully cancelled
       
       >>> # Try to cancel a non-existent booking
       >>> result = CancelFlightBooking("XYZ999")
       >>> print(result)
       there is no flight booking with id XYZ999
   
   Note:
       - The function operates on the global flight_booking dictionary
       - Once cancelled, the booking data is permanently removed from memory
       - No validation is performed on the flight_id format
       - The cancellation is immediate and cannot be undone
       - This function is decorated with @tool for use in agent workflows
   
   Side Effects:
       - Modifies the global flight_booking dictionary by removing entries
       - The cancelled booking data is permanently lost from the current session
   """
   if flight_number not in flight_booking:
       return f"there is no flight booking with id {flight_number}"
   else:
       del flight_booking[flight_id]
       return f"Flight booking with flight Id {flight_number} is successfully cancelled"



@tool 
def FlightBooking(flight_number : str , departure_airport_iata : str , total_number_seat : int , destination_airport_iata : str , seat_type = "economy"):
    """
       Book a flight ticket and store the booking details in the system.
       
       This function handles flight ticket booking by validating availability, storing booking
       information, and returning confirmation details. It checks for existing bookings on the
       same flight and prevents duplicate bookings.
       
       Args:
           flight_number (str): Unique identifier for the specific flight to be booked.
                           This should match a valid flight ID in the system.
           departure_airport_iata (str): IATA code of the departure airport 
                                       (e.g., 'DEL' for Delhi, 'BOM' for Mumbai).
           total_number_seat (int): Total number of seats/passengers to book on the flight.
                                  Must be a positive integer.
           destination_airport_iata (str): IATA code of the destination airport
                                         (e.g., 'BLR' for Bangalore, 'MAA' for Chennai).
           seat_type (str, optional): Type of seat class to book. Defaults to "economy".
                                    Common options: "economy", "business", "first".
       
       Returns:
           str: A formatted string containing either:
                - Success message with complete booking confirmation details including
                  flight ID, departure/destination airports, and seat type
                - Error message if the flight is already booked by the user
                - Exception error message if any technical issues occur during booking
       
       Raises:
           Exception: Catches and returns any unexpected errors that occur during the
                     booking process as a formatted error message string.
       
       Example:
           >>> result = FlightBooking("AI101", "DEL", 2, "BLR", "business")
           >>> print(result)
           ✅ Flight ticket booked successfully!
           
           Ticket Details:
           - Flight: AI101
           - From: DEL
           - To: BLR
           - Seat: business
           
           Thank you for booking with us!
           
           >>> # Attempting to book the same flight again
           >>> result = FlightBooking("AI101", "DEL", 1, "BLR")
           >>> print(result)
           You Already have a booking in this flight for 2 from DEL to BLR
       
       Note:
           - The function maintains booking state in the global flight_booking dictionary
           - Each flight_id can only have one active booking per user
           - IATA codes should be valid 3-letter airport codes
           - The function does not validate flight availability in the database
    """
    try:
        if flight_number  in flight_booking:
            return f"You Already have a booking in this flight for {flight_booking[flight_id]['num_of_people']} from {flight_booking[flight_id]['departure']} to {flight_booking[flight_id]['destination']}"
        info = {
            "departure" : departure_airport_iata,
            "num_of_people":total_number_seat,
            "destination":destination_airport_iata,
            "seat_type":seat_type
        }
        flight_booking[flight_number] = info
        
        return f"✅ Flight ticket booked successfully!\n\nTicket Details:\n- Flight: {flight_number}\n- From: {departure_airport_iata}\n- To: {destination_airport_iata}\n- \n- Seat: {seat_type}\n\nThank you for booking with us!"
    except Exception as e:
        return f"Error booking flight: {str(e)}"
    
@tool
def HotelSearch(
    country_code: str,
    city: str,
    num_of_hotels: Optional[int] = 5,
    longitude: Optional[str] = None,
    latitude: Optional[str] = None,
    radius: Optional[int] = 5000,
    ai_search: Optional[str] = None,
    zipcode: Optional[int] = None,
    min_rating: Optional[float] = None,
    min_review_count: Optional[int] = None,
    hotel_type_id: Optional[List[int]] = None,
    chain_id: Optional[List[int]] = None,
    strict_facilities: Optional[bool] = True
) -> Union[List[Dict[str, Any]], str]:
    """
    Search for available hotels based on specified criteria. This tool interacts with a hotel database
    to find hotels that match the given search criteria.

    Args:
        country_code (str): The ISO 3166-1 alpha-2 country code for the location.
        city (str): The city where you want to search for hotels.
        num_of_hotels (int, optional): The maximum number of hotels to return. Defaults to 5.
        longitude (str, optional): Longitude for location-based search.
        latitude (str, optional): Latitude for location-based search.
        radius (int, optional): Search radius in meters from lat/lon. Defaults to 5000.
        ai_search (str, optional): AI-generated search query or additional context or list of facilites.
        zipcode (int, optional): Postal code for precise location filtering.
        min_rating (float, optional): Minimum hotel rating (out of 5) .
        min_review_count (int, optional): Minimum number of reviews a hotel must have.
        hotel_type_id (List[int], optional): List of hotel type IDs to filter by.
        chain_id (List[int], optional): List of hotel chain IDs to filter by.
        strict_facilities (bool, optional): Whether to strictly require all specified facilities. Defaults to True.

    Returns:
        A list of dictionaries representing hotels, or an error message string.
    """
    try:
        base_url = f"https://api.liteapi.travel/v3.0/data/hotels?countryCode={country_code}&cityName={city}"

        # Add optional parameters to the URL
        if num_of_hotels is not None:
            base_url += f"&limit={num_of_hotels}"
        if longitude is not None and latitude is not None:
            base_url += f"&longitude={longitude}&latitude={latitude}"
        if radius is not None:
            base_url += f"&radius={radius}"
        if ai_search is not None:
            base_url += f"&aiSearch={requests.utils.quote(ai_search)}"
        if zipcode is not None:
            base_url += f"&zip={zipcode}"
        if min_rating is not None:
            base_url += f"&minRating={int(min_rating)}"
        if min_review_count is not None:
            base_url += f"&minReviewCount={min_review_count}"
        if hotel_type_id is not None:
            base_url += "&hotelTypeIds=" + ",".join(map(str, hotel_type_id))
        if chain_id is not None:
            base_url += "&chainIds=" + ",".join(map(str, chain_id))
        if strict_facilities:
            base_url += f"&strictFacilitiesFiltering={str(strict_facilities).lower()}"

        headers = {
            "accept": "application/json",
            "X-API-Key": secret_value_1 # Replace with your secret_value_1
        }

        response = requests.get(base_url, headers=headers)
        response.raise_for_status()  # This will raise an exception for HTTP errors (4xx or 5xx)
        data = response.json()

        # hotel_list = []
        # if "data" in data and data["data"]:
        #     for item in data["data"]:
        #         hotel_list.append({
        #             "name": item.get("name"),
        #             "description": item.get("hotelDescription"),
        #             "address": item.get("address"),
        #             "rating": item.get("starRating")
        #         })
        
        # if not hotel_list:
        #     return "No hotels found matching the criteria."

        return  data["data"][:5]

    except requests.exceptions.HTTPError as http_err:
        return f"HTTP error occurred: {http_err} - {response.text}"
    except Exception as e:
        return f"An error occurred while searching for hotels: {str(e)}"


@tool
def HotelTextSearch(user_txt : str):
    """
    Search for hotels using a free-text query via the LiteAPI Travel service.

    Parameters:
        user_txt (str): The user-provided search query describing hotels, destinations, or features.

    Returns:
        requests.Response: The response object from the API request containing the search results.
                          The response data can be accessed using .json() or .text.

    Example:
        response = HotelTextSearch("hotels in Paris with pool")
        hotels = response.json()
        print(hotels)

    Note:
        - The function constructs the search URL by splitting the input text and joining words with '%20'.
        - Requires a valid API key included in the headers for authentication.
        - The function returns the raw response from the requests library.
    """
    url = "https://api.liteapi.travel/v3.0/data/hotel/search?query="
    for i in user_txt.split(" "):
        url += i + "%20"
    url = url[:-3]
    headers = {
        "accept": "application/json",
        "X-API-Key": "sand_e23fbac5-1345-42ae-8cd9-c7a9cc64d03f"
    }
    
    response = requests.get(url, headers=headers)
    data = response.json()
    return data
    
@tool
def CancelBooking(hotel_id : str , hotel_name : str):
    """
    Cancel an existing hotel booking.

    Parameters:
        hotel_id (str): The unique identifier of the hotel to cancel the booking for.
        hotel_name (str): The name of the hotel.

    Returns:
        str: A message indicating whether the booking was cancelled or if no booking was found.
    
    Example:
        message = CancelBooking("12345", "Grand Palace")
        print(message)
    """
    if hotel_id not in hotel_booking :
        return f"You dont have any booking in hotel {hotel_name}"

    del hotel_booking[hotel_id]
    return f"booking cancled for hote {hotel_name}"

@tool
def BookHotel(hotel_id: str, hotel_name: str, num_of_people: int, check_in_date: str, check_out_date: Optional[str] = None):
    """
    Book a hotel and store the booking details.

    Parameters:
        hotel_id (str): Unique identifier for the hotel.
        hotel_name (str): Name of the hotel.
        num_of_people (int): Number of people for the booking.
        check_in_date (str): Check-in date for the booking (YYYY-MM-DD format).
        check_out_date (Optional[str]): Check-out date for the booking (YYYY-MM-DD format). If not provided, defaults to 'not decided'.

    Returns:
        str: Message indicating the booking status.
    """
    if hotel_id in hotel_bookings:
        return f"You already have a booking in the hotel {hotel_name} for {num_of_people} people."

    entry = {
        "HotelName": hotel_name,
        "NumberOfPeople": num_of_people,
        "CheckIn": check_in_date,
        "CheckOut": check_out_date if check_out_date is not None else "not decided"
    }

    hotel_booking[hotel_id] = entry
    return f"Booking successful at {hotel_name} for {num_of_people} people, check-in: {check_in_date}, check-out: {entry['CheckOut']}."


@tool
def HotelDetail(hotel_id : str):
    """
    Fetch detailed information about a specific hotel from the LiteAPI Travel service.

    Parameters:
        hotel_id (str): The unique identifier of the hotel to retrieve details for.

    Returns:
        dict: A dictionary containing detailed information about the hotel, as returned by the LiteAPI Travel API.
    """
    
    url = f"https://api.liteapi.travel/v3.0/data/hotel?hotelId={hotel_id}&timeout=4"
    headers = {
        "accept": "application/json",
        "X-API-Key": secret_value_1
        }
    
    response = requests.get(url, headers=headers)
    data = response.json()
    return data

# Enhanced model call to handle multimodal input
def model_call(state: AgentState) -> AgentState:
    system_prompt = SystemMessage(content="""
    You are an advanced Travel Assistant chatbot with comprehensive knowledge from Wikipedia and other travel sources.
    
    You have access to:
    1. Hotel search and booking tools (HotelSearch, HotelTextSearch, BookHotel, CancelBooking, HotelDetail)
    2. Flight search and booking tools (FlightSearch, FlightBooking, CancelFlightBooking)
    3. Comprehensive travel knowledge base including Wikipedia (TravelKnowledgeSearch for general queries)
    4. Destination-specific information (DestinationInfo for detailed destination info)
    5. Image understanding capabilities for visual travel inspiration
    
    Guidelines:
    - For general travel questions (visa, weather, tips, culture, attractions), use TravelKnowledgeSearch
    - For specific destination information, use DestinationInfo
    - For hotel/flight services, use the appropriate booking tools
    - If an image is provided, incorporate its context into your response
    - Always provide helpful, accurate, and personalized travel assistance
    - Include relevant tips and warnings when appropriate
    """)

    # Get chat history
    chat_history = langchain_memory.chat_memory.messages

    # Check if there's image context
    image_context_msg = []
    if state.get("image_context"):
        image_context_msg = [SystemMessage(content=f"User provided an image. {state['image_context']}")]

    # Combine all messages
    all_messages = [system_prompt] + chat_history + image_context_msg + state["messages"]

    response = llm.invoke(all_messages)
    tool_results = []

    # Execute tool calls
    if response.tool_calls:
        for tool_call in response.tool_calls:
            tool_name = tool_call['name']
            tool_args = tool_call['args']
            tool_function = globals().get(tool_name)
            if tool_function:
                try:
                    result = tool_function.invoke(tool_args)
                    tool_results.append(f"Tool Result ({tool_name}): {result}")
                except Exception as e:
                    tool_results.append(f"Error executing {tool_name}: {str(e)}")

    # Add tool results to messages
    for result in tool_results:
        response.content += f"\n{result}"

    # Save to memory
    if state["messages"]:
        last_user_message = state["messages"][-1]
        if isinstance(last_user_message, HumanMessage):
            langchain_memory.save_context(
                {"input": last_user_message.content},
                {"output": response.content}
            )

    return {"messages": [response]}

# Update tools list with all available tools
tools = [
    HotelSearch, 
    HotelDetail, 
    HotelTextSearch, 
    BookHotel, 
    CancelBooking, 
    FlightSearch, 
    FlightBooking, 
    CancelFlightBooking,
    TravelKnowledgeSearch,  # RAG tool
    DestinationInfo  # Destination-specific tool
]

llm = llm.bind_tools(tools)

# Graph setup
def should_continue(state: AgentState):
    message = state["messages"]
    last_message = message[-1]
    if not last_message.tool_calls:
        return "end"
    else:
        return "continue"

graph = StateGraph(AgentState)
graph.add_node("our_agent", model_call)

tool_node = ToolNode(tools=tools)
graph.add_node("tools", tool_node)

graph.set_entry_point("our_agent")
graph.add_conditional_edges("our_agent",
                            should_continue,
                           {
                               "continue": "tools",
                               "end": END,
                           },)
graph.add_edge("tools", "our_agent")
app = graph.compile()

# Enhanced stream printing function
def print_stream(stream, file):
    for s in stream:
        message = s["messages"][-1]
        
        # Directly execute and print tool results
        if hasattr(message, "tool_calls") and message.tool_calls:
            for tool_call in message.tool_calls:
                tool_name = tool_call['name']
                tool_args = tool_call['args']
                
                print(f"AI is calling tool: {tool_name} with args: {tool_args}")
                file.write(f"AI is calling tool: {tool_name} with args: {tool_args}\n")
                
                # Execute the tool function
                tool_function = globals().get(tool_name)
                if tool_function:
                    try:
                        result = tool_function.invoke(tool_args)
                        print(f"Tool Result ({tool_name}): {result}\n")
                        file.write(f"Tool Result ({tool_name}): {result}\n")
                    except Exception as e:
                        print(f"Error executing {tool_name}: {str(e)}\n")
                else:
                    print(f"Tool {tool_name} not found!\n")
        
        # Print regular AI messages
        elif isinstance(message, AIMessage):
            if message.content:
                print(f"AI Message: {message.content}")
                file.write(f"AI Message: {message.content}\n")
                
        # Print human messages
        elif isinstance(message, HumanMessage):
            print(f"\n{'='*80}\nHuman Message: {message.content}")
            file.write(f"\n{'='*80}\nHuman Message: {message.content}\n")

# Function to handle multimodal input
def process_user_input(text_input: str, image_input: Optional[Union[str, bytes, Image.Image]] = None) -> Dict:
    """
    Process user input that may include both text and image.
    
    Args:
        text_input: The text query from the user
        image_input: Optional image data (base64 string, bytes, or PIL Image)
        
    Returns:
        Dictionary containing messages and optional image context
    """
    state = {"messages": [HumanMessage(content=text_input)]}
    
    if image_input:
        image_description = process_image(image_input)
        state["image_context"] = image_description
    
    return state

def load_image_from_url(url: str) -> Image.Image:
    """
    Load an image from URL and return as PIL Image object.
    
    Args:
        url: URL of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url, timeout=10)
    response.raise_for_status()  # Raise exception for bad status codes
    image = Image.open(io.BytesIO(response.content))
    return image

def url_to_bytes(url: str) -> bytes:
    """
    Load an image from URL and return as bytes.
    
    Args:
        url: URL of the image
        
    Returns:
        Image as bytes
    """
    response = requests.get(url, timeout=10)
    response.raise_for_status()
    return response.content

def load_image_any_format(url: str, format_type: str = "pil") -> Union[str, bytes, Image.Image]:
    """
    Load an image from URL and return in specified format.
    
    Args:
        url: URL of the image
        format_type: "pil", "bytes", or "base64"
        
    Returns:
        Image in requested format
    """
    try:
        if format_type == "pil":
            return load_image_from_url(url)
        elif format_type == "bytes":
            return url_to_bytes(url)
        elif format_type == "base64":
            return url_to_base64(url)
        else:
            raise ValueError(f"Unknown format type: {format_type}")
    except Exception as e:
        raise Exception(f"Failed to load image from URL: {str(e)}")

def url_to_base64(url: str) -> str:
    """
    Load an image from URL and return as base64 encoded string.
    
    Args:
        url: URL of the image
        
    Returns:
        Base64 encoded string of the image
    """
    response = requests.get(url, timeout=10)
    response.raise_for_status()
    return base64.b64encode(response.content).decode('utf-8')

# Example usage with different types of queries
if __name__ == "__main__":
    # Create a file for logging
    with open("travel_assistant_log.txt", "w") as log_file:
        
        # Example 1: General travel query using Wikipedia knowledge
        print("\n=== Example 1: General Travel Query ===")
        inputs = {"messages": [HumanMessage(content="What are the top attractions in Jaipur and when is the best time to visit?")]}
        print_stream(app.stream(inputs, stream_mode="values"), log_file)

        langchain_memory.clear()
        time.sleep(5)
        # Example 2: Flight search
        print("\n=== Example 2: Flight Search ===")
        inputs = {"messages": [HumanMessage(content="Find flights from Delhi to Mumbai on June 6")]}
        print_stream(app.stream(inputs, stream_mode="values"), log_file)

        langchain_memory.clear()
        time.sleep(5)

        
        # Example 3: Destination information
        print("\n=== Example 3: Destination Information ===")
        inputs = {"messages": [HumanMessage(content="Tell me about Kerala - culture, attractions, and best time to visit")]}
        print_stream(app.stream(inputs, stream_mode="values"), log_file)

        langchain_memory.clear()
        time.sleep(5)
        
        # Example 4: Travel tips from knowledge base
        print("\n=== Example 4: Travel Tips ===")
        inputs = {"messages": [HumanMessage(content="What are the baggage rules for domestic flights in India?")]}
        print_stream(app.stream(inputs, stream_mode="values"), log_file)

        langchain_memory.clear()
        time.sleep(5)
        
        # Example 5: Multimodal input (if image provided)
        print("\n=== Example 5: Multimodal Query ===")
        # Simulate image input - in real use, this would be actual image data
        try:
            image_url = "https://images.unsplash.com/photo-1516483638261-f4dbaf036963?q=80&w=686&auto=format&fit=crop&ixlib=rb-4.1.0&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D"
            pil_image = load_image_from_url(image_url)
            state_with_image = process_user_input(
                "I want to visit a place like this. Can you help me find similar destinations?",
                pil_image  # Would be actual image data
            )
            print_stream(app.stream(state_with_image, stream_mode="values"), log_file)
        except Exception as e:
            print(f"Error processing image: {str(e)}")
            # Fallback to text-only query
            inputs = {"messages": [HumanMessage(content="Suggest some scenic mountain destinations in Europe")]}
            print_stream(app.stream(inputs, stream_mode="values"), log_file)



Loading existing vector store...
Vector store loaded successfully!

=== Example 1: General Travel Query ===

Human Message: What are the top attractions in Jaipur and when is the best time to visit?
AI is calling tool: TravelKnowledgeSearch with args: {'query': 'top attractions in Jaipur'}
Tool Result (TravelKnowledgeSearch): 

AI is calling tool: TravelKnowledgeSearch with args: {'query': 'best time to visit Jaipur'}
Tool Result (TravelKnowledgeSearch): 

AI Message: Here’s a detailed guide to **Jaipur’s top attractions** and the **best time to visit**:

---

### **Top Attractions in Jaipur**
1. **Amber Fort (Amber Palace)**  
   - A stunning hilltop fort with Mughal and Rajput architecture. Highlights include the Sheesh Mahal (Mirror Palace) and the Diwan-i-Aam.  
   - **Tip**: Take an elephant ride up the hill for panoramic views.

2. **City Palace**  
   - A majestic complex of palaces, museums, and gardens. The Diwan-i-Khas (Hall of Private Audience) and the Chandra Mahal are must

In [29]:
langchain_memory.clear()

In [ ]:
# Utility functions for the travel assistant

def search_similar_destinations(description: str) -> List[str]:
    """
    Search for destinations similar to a given description.
    """
    similar_query = f"destinations similar to {description} tourist places like {description}"
    docs = retriever.get_relevant_documents(similar_query)
    
    destinations = set()
    for doc in docs[:5]:
        # Extract destination names from the content
        content = doc.page_content
        # Simple extraction - in production, use NER or more sophisticated methods
        words = content.split()
        for i, word in enumerate(words):
            if word.capitalize() == word and len(word) > 3:
                destinations.add(word)
    
    return list(destinations)[:10]

def get_travel_season_info(destination: str) -> str:
    """
    Get seasonal travel information for a destination.
    """
    season_query = f"{destination} best time to visit weather seasons climate"
    docs = retriever.get_relevant_documents(season_query)
    
    if docs:
        return docs[0].page_content[:500]
    return "Seasonal information not available."

# Add batch processing capability for multiple queries
def batch_process_queries(queries: List[str], log_file):
    """
    Process multiple queries in batch.
    
    Args:
        queries: List of query strings
        log_file: File object for logging
    """
    for i, query in enumerate(queries, 1):
        print(f"\n=== Processing Query {i}/{len(queries)} ===")
        inputs = {"messages": [HumanMessage(content=query)]}
        print_stream(app.stream(inputs, stream_mode="values"), log_file)

# Network visualization (optional)
import networkx as nx
import matplotlib.pyplot as plt

def visualize_workflow():
    """Create a visual representation of the workflow graph"""
    G = nx.DiGraph()
    
    # Add nodes
    G.add_node("our_agent", color='lightblue', size=1000)
    G.add_node("tools", color='lightgreen', size=800)
    G.add_node("end", color='lightcoral', size=600)
    
    # Add edges
    G.add_edge("our_agent", "tools", label="continue")
    G.add_edge("our_agent", "end", label="end")
    G.add_edge("tools", "our_agent", label="process")
    
    # Draw the graph
    plt.figure(figsize=(10, 8))
    pos = nx.spring_layout(G)
    
    # Draw nodes
    node_colors = [G.nodes[node].get('color', 'lightblue') for node in G.nodes()]
    node_sizes = [G.nodes[node].get('size', 500) for node in G.nodes()]
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=node_sizes)
    
    # Draw edges and labels
    nx.draw_networkx_edges(G, pos, edge_color='gray', arrows=True, arrowsize=20)
    nx.draw_networkx_labels(G, pos)
    
    # Draw edge labels
    edge_labels = nx.get_edge_attributes(G, 'label')
    nx.draw_networkx_edge_labels(G, pos, edge_labels)
    
    plt.title("Travel Assistant Workflow Graph")
    plt.axis('off')
    plt.tight_layout()
    plt.savefig('travel_assistant_workflow.png', dpi=300, bbox_inches='tight')
    plt.show()

# Call visualization if needed
# visualize_workflow()

print("Travel Assistant with Wikipedia Knowledge Base initialized successfully!")
print(f"Vector store contains information from {len(vectorstore.index_to_docstore_id)} document chunks")